In [20]:
import numpy as np
from scipy.linalg import inv, det

# Step 1: Define the dataset
X = np.array([
    [1.0, 2.0],
    [1.5, 2.5],
    [3.0, 4.0],
    [3.5, 4.5]
])
Y = np.array([0, 0, 1, 1])

# Step 2: Compute parameters
# 2.1 Class priors
phi_1 = np.mean(Y)  # P(Y=1)
phi_0 = 1 - phi_1   # P(Y=0)

# 2.2 Class means
mu_0 = np.mean(X[Y == 0], axis=0)
mu_1 = np.mean(X[Y == 1], axis=0)

# 2.3 Shared covariance matrix
m = len(Y)
Sigma = np.zeros((2, 2))
for i in range(m):
    x_diff = X[i] - (mu_1 if Y[i] == 1 else mu_0)
    Sigma += np.outer(x_diff, x_diff)
Sigma /= m

# Check if Sigma is singular or nearly singular
if np.linalg.det(Sigma) == 0:
    print("Sigma is singular, adding regularization.")
    epsilon = 1e-6
    Sigma += epsilon * np.eye(Sigma.shape[0])  # Regularize with small identity matrix

# Step 3: Predict for a new point X_new using the Gaussian formula
X_new = np.array([2.5, 3.0])

# Define Gaussian formula components
Sigma_inv = inv(Sigma)
Sigma_det = det(Sigma)

def gaussian_pdf(x, mu, Sigma_inv, Sigma_det):
    diff = x - mu
    exponent = -0.5 * diff.T @ Sigma_inv @ diff
    normalization = 1 / (2 * np.pi * np.sqrt(Sigma_det))
    return normalization * np.exp(exponent)

# Compute likelihoods P(X | Y)
p_x_given_y0 = gaussian_pdf(X_new, mu_0, Sigma_inv, Sigma_det)
p_x_given_y1 = gaussian_pdf(X_new, mu_1, Sigma_inv, Sigma_det)

# Compute posteriors P(Y | X)
p_y0_given_x = p_x_given_y0 * phi_0
p_y1_given_x = p_x_given_y1 * phi_1

# Compute total probability
p_total = p_y0_given_x + p_y1_given_x

# Ensure p_total is not zero before normalizing
if p_total == 0:
    print("Warning: Total probability is zero, cannot normalize.")
    p_y0_given_x, p_y1_given_x = 0, 0  # or assign some default values
else:
    # Normalize to get probabilities
    p_y0_given_x /= p_total
    p_y1_given_x /= p_total

# Classification
prediction = 1 if p_y1_given_x > p_y0_given_x else 0

# Output results
print("Class priors:")
print(f"P(Y=0): {phi_0}")
print(f"P(Y=1): {phi_1}\n")

print("Class means:")
print(f"mu_0: {mu_0}")
print(f"mu_1: {mu_1}\n")

print("Shared covariance matrix:")
print(Sigma, "\n")

print("Likelihoods:")
print(f"P(X | Y=0): {p_x_given_y0}")
print(f"P(X | Y=1): {p_x_given_y1}\n")

print("Posterior probabilities:")
print(f"P(Y=0 | X): {p_y0_given_x}")
print(f"P(Y=1 | X): {p_y1_given_x}\n")

print("Prediction:")
print(f"Class: {prediction}")


Sigma is singular, adding regularization.
Class priors:
P(Y=0): 0.5
P(Y=1): 0.5

Class means:
mu_0: [1.25 2.25]
mu_1: [3.25 4.25]

Shared covariance matrix:
[[0.062501 0.0625  ]
 [0.0625   0.062501]] 

Likelihoods:
P(X | Y=0): 0.0
P(X | Y=1): 0.0

Posterior probabilities:
P(Y=0 | X): 0
P(Y=1 | X): 0

Prediction:
Class: 0


In [21]:
# exercise

In [30]:
import numpy as np
from scipy.linalg import inv, det

# data set
feature1 = np.array([1.0, 1.5, 3.0, 3.5])
feature2 = np.array([2.0, 2.5, 4.0, 4.5])
label = np.array([0, 0, 1, 1])

x_data = np.c_[feature1, feature2]
y_data = label

# parameters
# phi
phi_1 = np.mean(y_data)
phi_0 = 1 - phi_1

# mean vectors
mean_0 = np.mean(x_data[y_data == 0], axis=0)
mean_1 = np.mean(x_data[y_data == 1], axis=0)

# covariance matrix
Sigma = np.zeros((2, 2))
n = len(y_data)

for i in range(n):
    diff = x_data[i] - (mean_0 if y_data[i] == 0 else mean_1)
    Sigma += np.outer(diff, diff)

Sigma /= n

# Regularize if the covariance matrix is singular
if np.linalg.det(Sigma) == 0:
    print("Sigma is singular, adding regularization.")
    epsilon = 1e-6
    Sigma += epsilon * np.eye(Sigma.shape[0])  # Regularize with small identity matrix

# Determinant and inverse of the covariance matrix
Sigma_inv = inv(Sigma)
Sigma_det = det(Sigma)

# Gaussian Discriminative function for each sample
def GaussianDiscriminative(x_data, mean, Sigma_inv, Sigma_det):
    n = len(mean)  # number of features
    normalize = 1 / (np.sqrt((2 * np.pi) ** n * Sigma_det))  # normalization factor for Gaussian
    Diff = x_data - mean  # difference between data and mean
    exponent = -0.5 * np.sum(Diff @ Sigma_inv * Diff, axis=1)  # correct matrix multiplication for exponent
    return normalize * np.exp(exponent)

# Calculate the probabilities for each class
p_x_given_y0 = GaussianDiscriminative(x_data, mean_0, Sigma_inv, Sigma_det)
p_x_given_y1 = GaussianDiscriminative(x_data, mean_1, Sigma_inv, Sigma_det)

# Posterior probabilities for each class
posterior_y0 = p_x_given_y0 * phi_0
posterior_y1 = p_x_given_y1 * phi_1

# Compute total probability
totalProbability = posterior_y0 + posterior_y1

# Ensure totalProbability is not zero before normalizing
if np.any(totalProbability == 0):
    posterior_y0, posterior_y1 = 0, 0  # or assign some default values
else:
    # Normalize to get probabilities
    posterior_y0 /= totalProbability
    posterior_y1 /= totalProbability

# Make predictions based on the higher posterior probability
predictions = np.where(posterior_y1 > posterior_y0, 1, 0)

print(predictions)


Sigma is singular, adding regularization.
[0 0 1 1]


In [1]:
# exercise

In [4]:
import numpy as np
from scipy.linalg import inv, det

# dataset
Feature1 = np.arange(1,5)
Feature2 = np.array([2.0,3.0,3.5,4.5])
Ylabel = np.array([0,0,1,1])

X = np.c_[Feature1,Feature2]

# parameters
#phi
phi0 = np.mean(Ylabel)
phi1 = 1 - phi0

# mean
mu_0 = np.mean(X[Ylabel == 0], axis = 0)
mu_1 = np.mean(X[Ylabel == 1], axis = 0)

# sigma
SiGma = np.zeros((2,2))
examples = len(Ylabel)

for i in range(examples):
    difference = X[i] - (mu_0 if Ylabel[i] == 0 else mu_1)
    SiGma += np.outer(difference,difference)
SiGma /= examples

if np.linalg.det(SiGma) == 0:
    print("Sigma is singular, adding regularization.")
    epsilon = 1e-6
    SiGma += epsilon * np.eye(SiGma.shape[0])
    
SiGma_inv = inv(SiGma)
SiGma_det = det(SiGma)



# gda
def gaussianDA(X,mean,SiGma_inv,SiGma_det):
    nomalize =  1 / (np.sqrt((2 * np.pi) ** examples * SiGma_det))
    differen = X - mean
    exponet = -0.5 * differen.T @ SiGma_inv @ differen
    return nomalize * np.exp(exponet)

# new data
Xnew = np.array([3.5, 4.5])

# log liklihood
p_x_given_y0 = gaussianDA(Xnew,mu_0,SiGma_inv,SiGma_det)
p_x_given_y1 = gaussianDA(Xnew,mu_1,SiGma_inv,SiGma_det)

# posterior class
probability_y0 = p_x_given_y0 * phi0
probability_y1 = p_x_given_y1 * phi0

# total probability
totalProbability = probability_y0 + probability_y1

if totalProbability == 0:
    probability_y0,probability_y1 = 0,0

else:
    probability_y0 /= totalProbability
    probability_y1 /= totalProbability


PredictionClass = 1 if probability_y0 > probability_y1 else 0
print(f"New value belongs to class {PredictionClass}")


Sigma is singular, adding regularization.
New value belongs to class 1


In [5]:
# exercise 2

In [1]:
import numpy as np
from scipy.linalg import inv, det

# Dataset
Features_1 = np.linspace(1, 5, 5)
Features_2 = np.array([1.5, 2.5, 3.0, 4.0, 5.5])
Features_3 = np.array([0.5, 1.5, 2.0, 2.5, 3.0])
Labels = np.array([0, 0, 1, 1, 0])
featureX = np.c_[Features_1, Features_2, Features_3]

# Parameters
# Class priors
phis_0 = np.mean(Labels == 0)
phis_1 = 1 - phis_0

# Class means
mus_0 = np.mean(featureX[Labels == 0], axis=0)
mus_1 = np.mean(featureX[Labels == 1], axis=0)

# Covariance matrix
sig = np.zeros((featureX.shape[1], featureX.shape[1]))  # 3x3 matrix

length = len(Labels)
for i in range(length):
    dif = featureX[i] - (mus_0 if Labels[i] == 0 else mus_1)
    sig += np.outer(dif, dif)

sig /= length  # Normalize by N

# Ensure invertibility by adding a small regularization term
eps = 1e-6
sig += eps * np.eye(sig.shape[0])

# Compute inverse and determinant
sig_inv = inv(sig)
sig_det = det(sig)

# GDA function
def gda(featureX, sig_inv, sig_det, mean):
    d = featureX.shape[0]  # Number of features
    normalization = 1 / ((2 * np.pi) ** (d / 2) * np.sqrt(sig_det))
    diff = featureX - mean
    exp_term = -0.5 * diff.T @ sig_inv @ diff
    return normalization * np.exp(exp_term)

# New data (fixed to have 3 features)
data = np.array([3.0, 2.0, 1.5])

# Compute likelihood
p_x_given_Y0 = gda(data, sig_inv, sig_det, mus_0)
p_x_given_Y1 = gda(data, sig_inv, sig_det, mus_1)

# Compute posterior probability
p_Y0_given_x = p_x_given_Y0 * phis_0
p_Y1_given_x = p_x_given_Y1 * phis_1

# Total probability
TotalP = p_Y0_given_x + p_Y1_given_x

if TotalP == 0:
    p_Y0_given_x, p_Y1_given_x = 0, 0
else:
    p_Y0_given_x /= TotalP
    p_Y1_given_x /= TotalP

# Classification
predicted = 1 if p_Y1_given_x > p_Y0_given_x else 0

# Output results
print(f"Sigma:\n{sig}")
print(f"Means:\nClass 0: {mus_0}\nClass 1: {mus_1}")
print(f"Predicted class for the new feature is: {predicted}")


Sigma:
[[1.83333433 1.83333333 1.08333333]
 [1.83333333 1.83333433 1.08333333]
 [1.08333333 1.08333333 0.65833433]]
Means:
Class 0: [2.66666667 3.16666667 1.66666667]
Class 1: [3.5  3.5  2.25]
Predicted class for the new feature is: 0


In [2]:
# another ecxercise

In [8]:
import numpy as np
from scipy.linalg import inv, det

# dataset
featureA = np.array([2.0,3.0,4.5,5.0,3.5])
featureB = np.array([3.5,2.0,5.0,4.0,2.5])
featureC = np.array([1.0,1.5,2.5,2.0,1.2])
Ylabl = np.array([0,0,1,1,0])

x_features = np.c_[featureA,featureB,featureC]

# parameters
ph1 = np.mean(Ylabl == 1)
ph0 = 1 - ph1

#mean
mu0 = np.mean(x_features[Ylabl == 0], axis = 0)
mu1 = np.mean(x_features[Ylabl == 1], axis = 0)

# sigma
covariance = np.zeros((x_features.shape[1],x_features.shape[1]))
m_examples = len(Ylabl)

for i in range(m_examples):
    diffX = x_features[i] - (mu0 if Ylabl[i] == 0 else mu1)
    covariance += np.outer(diffX,diffX)
covariance /= m_examples

if np.linalg.det(covariance) == 0:
    eps = 1e-6
    covariance += eps + np.eye(x_features.shape[0])

covariance_inv = inv(covariance)
covariance_det = det(covariance)

# gausian
def gaussianDiscriminativeA(inputX,means,covariance_inv,covariance_det):
    d = x_features.shape[1]
    nomalize = 1 / (((2 * np.pi) ** (d / 2)) * covariance_det ** 0.5)

    x_mu_difference = inputX - means
    exponet = -0.5 * x_mu_difference.T @ covariance_inv @  x_mu_difference
    result = nomalize * np.exp(exponet)
    return result

# ne wdata
new_inputX = np.array([4.0,3.0,1.8])

# liklihood
mu0 = np.mean(x_features[Ylabl == 0], axis = 0)
probability_x_given_y0 = gaussianDiscriminativeA(new_inputX,mu0,covariance_inv,covariance_det)
probability_x_given_y1 = gaussianDiscriminativeA(new_inputX,mu1,covariance_inv,covariance_det)

# posterior class
posterior_y0 = probability_x_given_y0 * ph0
posterior_y1 = probability_x_given_y1 * ph1

# total probablity
ProbabilityTotal =posterior_y0 + posterior_y1

if ProbabilityTotal == 0:
    posterior_y0 , posterior_y1 = 0,0

else:
    posterior_y0 /= ProbabilityTotal
    posterior_y1 /= ProbabilityTotal

predictions = 1 if posterior_y1 > posterior_y0 else 0
print(f'Sigma: {covariance}\nMean0: {mu0}\nMean1: {mu1}\nPredicted class belongs to class {predictions}')



Sigma: [[ 0.25833333 -0.23333333  0.01833333]
 [-0.23333333  0.33333333 -0.02333333]
 [ 0.01833333 -0.02333333  0.05033333]]
Mean0: [2.83333333 2.66666667 1.23333333]
Mean1: [4.75 4.5  2.25]
Predicted class belongs to class 0


In [9]:
# loan prdeiction

In [20]:
import numpy as np
from scipy.linalg import inv,det

# dataset
AnnualIncome = np.array([50,60,80,90,55]) * 1000
LoanAmount = np.array([10,15,30,40,12]) * 1000
CreditScore = np.array([750,700,650,600,720])

default = np.array([0,0,1,1,0])
Inputs = np.c_[AnnualIncome,LoanAmount,CreditScore]

# parameters
Phi_1 = np.mean(default == 1)
Phi_0 = 1 - Phi_1

# means
Mu_0 = np.mean(Inputs[default == 0], axis = 0)
Mu_1 = np.mean(Inputs[default == 1], axis = 0)

# sigma
sigma_cov = np.zeros((Inputs.shape[1], Inputs.shape[1]))
examples_m = len(default)
for i in range(examples_m):
    difference_inputs_mus = Inputs[i] - (Mu_0 if default[i] == 0 else Mu_1)
    sigma_cov += np.outer(difference_inputs_mus,difference_inputs_mus)

sigma_cov /= examples_m

if np.linalg.det(sigma_cov) == 0:
    epsilion = 1e-6
    sigma_cov += epsilion + np.eye(Inputs.shape[1])

sigma_cov_inv = inv(sigma_cov)
sigma_cov_det = det(sigma_cov)

# gda
def gaussianDisriminative(data,Mean,sigma_cov_inv,sigma_cov_det):
    r = Inputs.shape[1]
    nomaliz = 1 / (((2 * np.pi**r/2)) * sigma_cov_det**0.5)
    diff_data_Mean = data - Mean
    expon = -0.5 * diff_data_Mean.T @ sigma_cov_inv @ diff_data_Mean
    results = nomaliz * np.exp(expon)
    return results

# new data
new_Inputs = np.array([75000,25000,670])

# likelihood
probability_inputs_given_y0 = gaussianDisriminative(new_Inputs,Mu_0,sigma_cov_inv,sigma_cov_det)
probability_inputs_given_y1 = gaussianDisriminative(new_Inputs,Mu_1,sigma_cov_inv,sigma_cov_det)

# posteriro
posterior_y0 = probability_inputs_given_y0 * Phi_0
posterior_y1 = probability_inputs_given_y1 * Phi_1

# total p
p_total = posterior_y0 + posterior_y1

if p_total == 0:
    posterior_y0 , posterior_y1 = 0,0

else:
    posterior_y0 /= p_total
    posterior_y1 /= p_total

predictedDefault = 1 if posterior_y1 > posterior_y0 else 0


print(f'Mean for class not default : {Mu_0}\nMean for class  default: {Mu_1}\nSigma: {sigma_cov}')
if predictedDefault ==1:
    print("The new customer will Default")

else:
    print("The new customer will not Default")




Mean for class not default : [55000.         12333.33333333   723.33333333]
Mean for class  default: [85000. 35000.   625.]
Sigma: [[ 2.00000000e+07  1.50000000e+07 -1.00000000e+05]
 [ 1.50000000e+07  1.25333333e+07 -7.46666667e+04]
 [-1.00000000e+05 -7.46666667e+04  5.03333333e+02]]
The new customer will Default


In [21]:
# credit card fraud detection

In [25]:
import numpy as np
from scipy.linalg import inv,det

# dataset
TransactionAmount = np.array([200,150,5000,4500,100]) # in dollars
Time_since_last_transact = np.array([120,90,10,5,180]) # time in min
Transaction_last24hrs = np.array([5,3,20,18,2])

labeld = np.array([0,0,1,1,0])

featuresData = np.c_[TransactionAmount,Time_since_last_transact,Transaction_last24hrs]

# parameters
Phis_1 = np.mean(labeld == 1)
Phis_0 = 1 - Phis_1

# means
Mus_0 = np.mean(featuresData[labeld == 0], axis = 0)
Mus_1 = np.mean(featuresData[labeld == 1], axis = 0)

# sigma
sigmaCovariance = np.zeros((featuresData.shape[1],featuresData.shape[1]))
n = len(labeld)

for i in range(n):
    data_diff = featuresData[i] - (Mus_0 if labeld[i] == 0 else Mus_1)
    sigmaCovariance += np.outer(data_diff,data_diff)
    
sigmaCovariance /= n

if np.linalg.det(sigmaCovariance) == 0:
    sigmaCovariance += 1e-6 + np.eye(featuresData.shape[0])

sigmaCovariance_inv = inv(sigmaCovariance)
sigmaCovariance_det = det(sigmaCovariance)

# gaussian
def gaussianAnalysis(new_data,mean,sigmaCovariance_inv,sigmaCovariance_det):
    a = featuresData.shape[1]
    Differe = new_data - mean
    nomali = 1 / (((2*np.pi**a/2))*sigmaCovariance_det**0.5)
    exp = -0.5 * Differe.T @ sigmaCovariance_inv @ Differe
    final_result = nomali * np.exp(exp)
    return final_result

# new data info
new_data = np.array([3000,15,10])

# likelihood
pro_data_y0 = gaussianAnalysis(new_data,Mus_0,sigmaCovariance_inv,sigmaCovariance_det)
pro_data_y1 = gaussianAnalysis(new_data,Mus_1,sigmaCovariance_inv,sigmaCovariance_det)

# posterior
posterior_y0_givendata = pro_data_y0 * Phis_0
posterior_y1_givendata = pro_data_y1 * Phis_1

# total probability
proTotal = posterior_y0_givendata + posterior_y1_givendata

if proTotal == 0:
    posterior_y0_givendata , posterior_y1_givendata = 0,0

else:
    posterior_y0_givendata /= proTotal
    posterior_y1_givendata /= proTotal

predictedData = 1 if posterior_y1_givendata > posterior_y0_givendata else 0

if predictedData == 1:
    print("This is a fraudulent transaction")
else:
    print("This is a legitimate transaction")
    


This is a fraudulent transaction
